# Naver 리뷰 데이터 전처리 
- train : https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
- test : https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

# 데이터 준비

In [2]:
# urllib -> urlopen -> urlretrieve
from urllib.request import urlretrieve

filename = "../data/ratings_test.txt"
url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"

ret = urlretrieve(url, filename)
ret # 파일을 확인하면 html 파일임 

('../data/ratings_test.txt', <http.client.HTTPMessage at 0x1fb218dd910>)

In [3]:
import pandas as pd
reviewDF = pd.read_table(filename, usecols=[1, 2])
reviewDF.head()

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


# 2. 데이터 전처리 
- 결측치, 중복 데이터
- 분류이기 때문에 클래스 균형 맞는지 확인 
- 텍스트 데이터 전처리 => 정제(불용어, 노이즈 제거), 토큰화, 2차 정제 => 단어사전 
- 텍스트 데이터 인코딩
- 텍스트 데이터 패딩

In [4]:
# # 결측치 확인
# reviewDF.dropna(inplace=True)
# reviewDF.info()

In [5]:
# 중복 행 제거
reviewDF.drop_duplicates(inplace=True)
reviewDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49206 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  49204 non-null  object
 1   label     49206 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [6]:
# 클래스 균형 확인
reviewDF["label"].value_counts() # 이만하면 균형...

label
1    24735
0    24471
Name: count, dtype: int64

In [7]:
# 피쳐와 레이블 분리
documentDF = reviewDF["document"].to_frame()
labelSR = reviewDF["label"]

print(f"documentDF.shape: {documentDF.shape}")
print(f"targetSR.shape: {labelSR.shape}")

documentDF.shape: (49206, 1)
targetSR.shape: (49206,)


In [8]:
# 노이즈 데이터 기준 : 한글만?

In [9]:
# # 구두점 제거
# import string, re
# 
# punc = string.punctuation
# 
# def remove_punctuation(text):
#     # 정규 표현식을 사용하여 구두점 제거
#     punctuation_pattern = '[' + re.escape(string.punctuation) + ']'
#     text = re.sub(punctuation_pattern, '', text)
#     return text
# 
# # document 컬럼에 구두점 제거 함수 적용
# documentDF['document'] = documentDF['document'].apply(remove_punctuation)
# 
# # 결과 출력
# print(documentDF)

In [10]:
import string

stop_word = string.punctuation # 조사들 추가해야함
hangule_pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"

In [11]:
# Lemmatization이 더 정확함(기본 사전형 단어 의미)

In [12]:
documentDF["document"] = documentDF["document"].str.replace(hangule_pattern, "",regex=True)
documentDF[:5]

,document
0,굳 ㅋ
1,
2,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠


In [13]:
import numpy as np

# regex = True => 정규식 적용
documentDF["document"] = documentDF["document"].str.replace("^ +", "", regex=True) # 스페이스는 몇개가오든 포함 안함
documentDF["document"].replace("",np.nan, inplace=True)
print(documentDF.isnull().sum())

document    315
dtype: int64


https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt

stopword 파일로 저장하기

In [14]:
documentDF.dropna(inplace=True)
documentDF["document"].isnull().sum()

0

In [15]:
hangul_stopword = "https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt"

In [16]:
urlretrieve(hangul_stopword, "../data/stop_word.txt")

('../data/stop_word.txt', <http.client.HTTPMessage at 0x1fb229a80d0>)

In [17]:
# 토큰 나누기
# 오크, 키위, 뽀로로...
from konlpy.tag import Okt  

# 형태소 분석기 인스턴스 생성
okt = Okt()

# 문장 합치기 (반복문 / 파일에 다 쓰기 ... 뭐 아무렇게나 통으로 -> 왜? Okt가 str외에 다른 형태는 안받음)
vocab = {}
for idx in range(documentDF.shape[0]):
    result = okt.morphs(documentDF.iloc[idx][0])
    for word in result:
        if len(word) >= 2:
            if word in vocab.keys():
                vocab[word] += 1
            else:
                vocab[word] = 1
print(vocab)

{'평점': 2213, '나쁘진': 12, '않지만': 46, '더욱': 94, '아니잖아': 29, '지루하지는': 4, '않은데': 24, '완전': 813, '막장': 228, '주고': 197, '보기': 552, '에는': 355, '아니었어도': 6, '다섯': 36, '줬을텐데': 2, '나와서': 133, '심기': 1, '불편하게': 10, '하죠': 7, '음악': 390, '주가': 16, '최고': 1950, '영화': 17234, '진정한': 108, '쓰레기': 1068, '마치': 68, '미국': 216, '애니': 293, '에서': 2331, '튀어나온듯': 1, '창의력': 2, '없는': 1200, '로봇': 36, '디자인': 19, '부터가': 27, '고개': 13, '젖게': 2, '한다': 461, '갈수록': 189, '개판': 59, '되가는': 1, '중국영화': 28, '유치하고': 74, '내용': 1347, '없음': 304, '잡다': 2, '끝남': 23, '안되는': 192, '무기': 25, '유치한': 58, '남무': 1, '그립다': 26, '동사서독': 2, '같은': 853, '이건': 876, '류작': 24, '이다': 1650, '이별': 20, '아픔': 40, '찾아오는': 3, '새로운': 86, '인연': 20, '기쁨': 7, '모든': 398, '사람': 1641, '그렇지는': 3, '않네': 14, '괜찮네요': 29, '오랜': 261, '포켓몬스터': 4, '잼밌': 1, '한국': 450, '독립영화': 69, '한계': 50, '그렇게': 215, '아버지': 123, '된다와': 1, '비교': 154, '청춘': 66, '아름답다': 62, '아름다': 42, '이성': 12, '흔들어': 4, '놓는다': 2, '찰나': 4, '포착': 2, '섬세하고': 10, '아름다운': 256, '수채화': 6, '퀴어': 8, '보이는': 62, '반전': 480,

In [176]:
len(vocab)

52788

In [177]:
# 불용어 제거
stopwords_path = "../data/stop_word.txt"

# 파일 열기
with open(stopwords_path, encoding="utf-8") as f:
    stopwords = f.readlines()
stopwords = [word.strip() for word in stopwords]

# 데이터 제거
for sword in stopwords:
    if sword in vocab.keys():
        vocab.pop(sword)

In [178]:
len(vocab)

52576

In [179]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)

[('영화', 17234),
 ('너무', 3735),
 ('정말', 3241),
 ('진짜', 2927),
 ('평점', 2213),
 ('연기', 2161),
 ('최고', 1950),
 ('생각', 1824),
 ('스토리', 1696),
 ('이다', 1650),
 ('드라마', 1642),
 ('사람', 1641),
 ('감동', 1631),
 ('하는', 1574),
 ('보고', 1551),
 ('하고', 1487),
 ('감독', 1411),
 ('배우', 1401),
 ('ㅋㅋ', 1384),
 ('그냥', 1351),
 ('내용', 1347),
 ('재미', 1306),
 ('보다', 1303),
 ('없는', 1200),
 ('봤는데', 1088),
 ('쓰레기', 1068),
 ('사랑', 1061),
 ('작품', 1007),
 ('다시', 954),
 ('없다', 931),
 ('마지막', 897),
 ('ㅠㅠ', 889),
 ('이건', 876),
 ('같은', 853),
 ('정도', 847),
 ('있는', 836),
 ('좋은', 820),
 ('완전', 813),
 ('처음', 813),
 ('장면', 804),
 ('주인공', 800),
 ('ㅋㅋㅋ', 799),
 ('이렇게', 787),
 ('입니다', 786),
 ('액션', 781),
 ('최악', 755),
 ('보는', 753),
 ('이야기', 746),
 ('지금', 738),
 ('ㅡㅡ', 719),
 ('연출', 707),
 ('없고', 705),
 ('느낌', 679),
 ('봐도', 677),
 ('별로', 672),
 ('인데', 662),
 ('재밌게', 656),
 ('역시', 652),
 ('많이', 648),
 ('명작', 648),
 ('이해', 642),
 ('라고', 633),
 ('때문', 619),
 ('이영화', 610),
 ('여자', 609),
 ('해서', 581),
 ('보면', 581),
 ('보기', 552),
 ('인생',

In [180]:
vocab["영화"], vocab["너무"]

(17234, 3735)

In [181]:
# 전처리 함수
# 토큰화 함수 
# <UNK> -> 0번
# <PADDING> -> 1번
# 길이보고 잘라내는 것까지 

In [182]:
# 데이터프레임으로 변환
vocabDF = pd.DataFrame.from_dict(vocab, orient="index")

vocabDF

,0
평점,2213
나쁘진,12
않지만,46
더욱,94
아니잖아,29
...,...
박약,1
닮았고,1
이등병,1
높았지만,1


In [183]:
vocabDF = pd.DataFrame(vocabDF.sort_values(by=0, ascending=False))
vocabDF

,0
영화,17234
너무,3735
정말,3241
진짜,2927
평점,2213
...,...
잡는것밖에,1
키얼스틴던스트,1
마인,1
물어봐도,1


# 3. 데이터셋용 단어사전/어휘사전 생성
- 최종 사용할 단어 수
- 특별한 의미의 문자 추가 : "<UNK>", "<PAD>"

In [184]:
VOCAB_DICT = {0:"<UNK>",
              1:"<PAD>"}

for idx in range(10):
    # print(vocabDF.iloc[idx])
    VOCAB_DICT[idx+2] = vocabDF.index[idx]
    
VOCAB_DICT # 2 : 단어1, 3: 단어2

{0: '<UNK>',
 1: '<PAD>',
 2: '영화',
 3: '너무',
 4: '정말',
 5: '진짜',
 6: '평점',
 7: '연기',
 8: '최고',
 9: '생각',
 10: '스토리',
 11: '이다'}

In [185]:
# 문장을 수치화 진행 -> 인코딩
test = ["영화", "우히힛", "최고", "생각"]

sentence = []
for tt in test:
    sentence.append(0) # 없으면 왜 0 -> UNK 
    for k, v in VOCAB_DICT.items():
        if v == tt:
            sentence[-1] = k
            break
sentence  

[2, 0, 8, 9]

In [186]:
# 수치값을 문자열로 변환 -> 디코딩
words = []
for tt in sentence:
    words.append(VOCAB_DICT.get(tt))
    
print(words)

['영화', '<UNK>', '최고', '생각']
